In [4]:
import dask.dataframe as dd

from py.consts import DATA_DIR
from py.utils import one_hot_encode_all


partition_size = "100MB"

In [5]:
dataset = dd.read_parquet(DATA_DIR / "prepared_y.parquet")
dataset = dataset.repartition(partition_size=partition_size)

print(dataset.npartitions)
print(dataset.divisions)
print(dataset.map_partitions(len).compute())

print()
print(dataset.head())

4
(None, None, None, None, None)
0    1096301
1    1096302
2    1096301
3    1096302
dtype: int64
      wegnummer   zijde     tijd_van_melden         laatste_activiteit  y  \
53100       A16  rechts 2020-01-10 07:21:00 2020-01-10 13:50:25.836719  1   
91510      N313   beide 2022-12-12 09:44:43 2022-12-12 14:50:21.076660  1   
50223       A15  rechts 2019-09-25 10:35:11 2019-09-25 16:20:04.022813  1   
4439       N351   beide 2016-04-13 18:39:47 2016-04-13 23:51:56.760000  1   
34728      N289   beide 2018-06-14 10:01:00 2018-06-14 18:30:03.738831  1   

           month        day  
53100    January     Friday  
91510   December     Monday  
50223  September  Wednesday  
4439       April  Wednesday  
34728       June   Thursday  


In [6]:
dataset["start_hour"] = dataset.tijd_van_melden.dt.hour
dataset = dataset.drop("tijd_van_melden", axis=1)
dataset["stop_hour"] = dataset.laatste_activiteit.dt.hour
dataset = dataset.drop("laatste_activiteit", axis=1)

dataset.compute()


,wegnummer,zijde,y,month,day,start_hour,stop_hour
53100,A16,rechts,1,January,Friday,7,13
91510,N313,beide,1,December,Monday,9,14
50223,A15,rechts,1,September,Wednesday,10,16
4439,N351,beide,1,April,Wednesday,18,23
34728,N289,beide,1,June,Thursday,10,18
...,...,...,...,...,...,...,...
6367,N844,beide,0,December,Friday,18,23
6365,N804,links,0,December,Friday,0,23
6367,N844,links,0,December,Friday,0,23
6365,N804,rechts,0,December,Friday,0,23


In [7]:
# zijde = one_hot_encode(df=dataset, column="zijde", dtype=int)
# day_of_week = one_hot_encode(df=dataset, column="day", dtype=int)
# month_of_year = one_hot_encode(df=dataset, column="month", dtype=int)
# road = one_hot_encode(df=dataset, column="wegnummer", dtype=int)
# # dataset = one_hot_encode(df=dataset, column="type_controle")
# # dataset = one_hot_encode(df=dataset, column="hm_paal")
# start_hour = one_hot_encode(df=dataset, column="start_hour", dtype=int)
# stop_hour = one_hot_encode(df=dataset, column="stop_hour", dtype=int)

dataset = one_hot_encode_all(ddf=dataset, columns=["zijde", "month", "day", "wegnummer", "start_hour", "stop_hour"], dtype=int)

# dataset.compute()

/Users/simonverhoek/Documents/programming/projects/flitsers/venv/lib/python3.11/site-packages/dask/dataframe/multi.py:1297: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(
/Users/simonverhoek/Documents/programming/projects/flitsers/venv/lib/python3.11/site-packages/dask/dataframe/multi.py:1297: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(
/Users/simonverhoek/Documents/programming/projects/flitsers/venv/lib/python3.11/site-packages/dask/dataframe/multi.py:1297: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(
/Users/simonverhoek/Documents/programming/projects/flitsers/venv/lib/python3

In [8]:
import dask

from py.utils import write_partition_to_parquet
from dask.diagnostics import ProgressBar

with ProgressBar():
    delayed_tasks = []
    for i, partition in enumerate(dataset.to_delayed()):
        delayed_task = dask.delayed(write_partition_to_parquet)(partition, i)
        delayed_tasks.append(delayed_task)
    
    # Execute the tasks and track progress
    results = dask.compute(*delayed_tasks)

# dataset.to_parquet(DATA_DIR / f"encoded_test.parquet", memory_limit=16e9)
# dataset.compute()

[######################################  ] | 95% Completed | 65.76 smspartition:          y  zijde_links  zijde_rechts  month_August  month_December  \
53100    1            0             1             0               0   
91510    1            0             0             0               1   
50223    1            0             1             0               0   
4439     1            0             0             0               0   
34728    1            0             0             0               0   
...     ..          ...           ...           ...             ...   
1049590  0            0             1             1               0   
1049591  0            0             0             1               0   
1049592  0            1             0             1               0   
1049593  0            0             1             1               0   
1049594  0            0             0             1               0   

         month_February  month_January  month_July  month_June  mo

In [ ]:
dups = dataset.map_partitions(lambda d: d.duplicated())
duplicates = dataset[dups].compute().index.tolist()
print(duplicates)

In [9]:
dataset.to_parquet(DATA_DIR / f"encoded_test.parquet")